# Homework 9

In [ ]:
from pint import UnitRegistry
ur = UnitRegistry()

In [ ]:
from CoolProp.CoolProp import PhaseSI, PropsSI
import CoolProp as CP

In [ ]:
import numpy as np

## 5.11
Chlorine gas leaving a reactor is pressurized and sent back to the reactor. The reactor inlet is 1.6 atm, so we need to know the pressure drop in order to know how much the recycle needs to be pressurized to overcome the losses in the piping run back to the reactor inlet.

Known
1. Reactor outlet is 200 deg. C and 1.5 atm.
2. Reactor inlet is 1.6 atm.
3. Mass flow rate is 4 lbm/min.
4. Pipe length is 80 feet.
5. Fittings include 6 elbows and 2 globe valves.

Elbows have $K_{elb} = 0.75$. Globe valves have $K_{gv} = 0.17$.

I'm going to use the Shackham equations for friction loss in the pipe.

$$ f = \left\{-1.737 \ln \left[0.269\frac{\epsilon}{D_i} - \frac{2.185}{Re}\ln\left(0.269\frac{\epsilon}{D_i} + \frac{14.5}{Re}\right)\right]\right\}^{-2} $$

### Define Friction Factor Function

In [ ]:
def ffric_shackham(e,D,Re):
# if I'm using pint I will need to make sure to send magnitudes to this function
    eByD = e/D;
    A = 0.269 * eByD;
    B = 2.185 / Re;
    C = 14.5 / Re;
#     print([A, B, C])
    f = (-1.737 * np.log(A - B*np.log(A + C)))**(-2) # np.log is natural log
    return f    

Test the function on some values ...

In [ ]:
ffric = ffric_shackham(0.00015,1/12,40000)
print(ffric)

0.00667541657832663


### Get Properties
The pressure change is small for a gas, so I'm going to assume an average pressure of 1.55 atm to get properties at 200 deg. C.

In [ ]:
CP.CoolProp.get_fluid_param_string("Chlorine","JSON")

RuntimeError: key [Chlorine] was not found in string_to_index_map in JSONFluidLibrary

Oh no ... CoolProp doesn't have chlorine ... so, I asked chatGPT for a python script to use WolframAlpha to query fluid properties ...

First, to use the Wolfram Alpha API and make queries in your Python code, you need to have an App ID. Here's how to get one:

- Go to the Wolfram Alpha developer portal.
- Sign in or create an account if you don't have one already.
- Click on the "Get an AppID" button.
- Fill out the form with your app's details, such as the name and description.
- Click on the "Create AppID" button.
- Your new App ID will be displayed on the page.
- Copy your App ID and use it in your Python code to make queries to the Wolfram Alpha API.

In [ ]:
import wolframalpha
# I had to do >> `pip install wolframalpha` first

app_id = "32XYTH-8WAQ64Y9KH" # this is my app_id ... yours would be different
client = wolframalpha.Client(app_id)

query = "density of chlorine at 200 degC and 1.55 atm"
res = client.query(query)

answer = next(res.results).text
print(answer)  # output: "997.047 kg/m^3 (kilograms per cubic meter)"

2.845 kg/m^3 (kilograms per cubic meter)


In [ ]:
query = "viscosity of chlorine at 200 degC and 1.55 atm"
res = client.query(query)

answer = next(res.results).text
print(answer)  # output: "997.047 kg/m^3 (kilograms per cubic meter)"

(data not available)


That still didn't work, so I just went to [Engineer's Toolbox](https://www.engineeringtoolbox.com/gases-absolute-dynamic-viscosity-d_1888.html).

$ \mu = 2.10e^{-5} $ Pa$\cdot$s 

In [ ]:
T_K = ur.Quantity(200, ur.degC).to(ur.degK);
P_Pa = ur.Quantity(1.55, ur.atm).to(ur.Pa);
rho = ur.Quantity(2.845, ur.kilogram / ur.meter**3);
mu = ur.Quantity(0.000021, ur.Pa * ur.second);
mDot = ur.Quantity(4,ur.lb/ur.minute).to(ur.kilogram/ur.second)
f"{mDot:~P}"

'0.030239491333333337 kg/s'

In [ ]:
query = "speed of sound for chlorine at 200 degC and 1.55 atm"
res = client.query(query)

answer = next(res.results).text
print(answer)  

964.6 km/h (kilometers per hour)


In [ ]:
c = ur.Quantity(964.6,ur.kilometer/ur.hour).to(ur.meter_per_second)
f"{c:~P}"

'267.94444444444446 mps'

In [ ]:
Mach = 0.2;
Vtarget = c * Mach;
print(Vtarget)

53.588888888888896 meter_per_second


In [ ]:
D = (4*mDot/np.pi/rho/Vtarget)**(0.5)
[print(D.to(ur.meter)), print(D.to(ur.feet)), print(D.to(ur.inch))]

0.015891459611367335 meter
0.0521373346829637 foot
0.6256480161955644 inch


[None, None, None]

In [ ]:
Re = rho * Vtarget * D / mu;
Re.ito(ur.meter/ur.meter)
print(Re)

115372.291064816 dimensionless


In [ ]:
e = ur.Quantity(0.00015,ur.feet).to(ur.meter);
ffric = ffric_shackham(e.magnitude,D.to(ur.meter).magnitude,Re)
print(ffric)

0.006753106756567064 dimensionless


In [ ]:
L = ur.Quantity(80,ur.feet).to(ur.meter);
D.ito(ur.meter); # just make sure the units for L and D are the same.
K_elb = 0.75;
K_gv = 0.17;
delPbyRho = (4*ffric*L/D + 6*K_elb + 2*K_gv)*(Vtarget**2/2)
print(delPbyRho)

66464.38490027997 meter_per_second ** 2


In [ ]:
delP = delPbyRho * rho
print(delP.to(ur.Pa)),print(delP.to(ur.psi))

189091.17504129652 pascal
27.425356252736652 pound_force_per_square_inch


(None, None)

In [ ]:
def delP_calc(D,mdot,rho,mu,epsilon,L):
    Vt = 4 * mdot / np.pi / rho / D**2;
    Re = rho * Vt * D / mu;
    ffric = ffric_shackham(epsilon,D,Re);
    K_elb = 0.75;
    K_gv = 0.17;
    delPbyRho = (4*ffric*L/D + 6*K_elb + 2*K_gv)*(Vt**2/2);
    delP = delPbyRho * rho;
    return delP, Vt    

In [ ]:
dP, Vt = delP_calc(ur.Quantity(2,ur.inch).to(ur.meter),
              mDot.to(ur.kilogram/ur.second),
              rho.to(ur.kilogram/ur.meter**3),
              mu.to(ur.Pa*ur.second),
              ur.Quantity(0.00015,ur.feet).to(ur.meter),
              ur.Quantity(80,ur.feet).to(ur.meter))
print(dP.to(ur.psi)), print(Vt.to(ur.meter_per_second))

0.09552861474024181 pound_force_per_square_inch
5.24414757760343 meter_per_second


(None, None)